In [1]:
import sys
sys.path.append('../')

In [2]:
from app.prediction import OperationsOnWiggle as ops

In [3]:
import glob
import os
import pandas as pd

In [4]:
conditions =  ["ELP", "MLP", "Stat"]

In [5]:
directory = "data_bacteroides/Subset_OnlyChrom"

In [6]:
pattern_forward = os.path.join(directory, '*forward*')

# Forward training on Bacteroides 

In [7]:
files = glob.glob(pattern_forward)

df_fordward = {}

for condition in conditions:
    file_list = []
    for file_path in files:
        if(condition in str(file_path)):
            file_list += [file_path]
    print(file_list)
    df_fordward[condition] = ops.parse_for_prediction(file_list)

['data_bacteroides/Subset_OnlyChrom\\ELP_S1_P1_forward.wig', 'data_bacteroides/Subset_OnlyChrom\\ELP_S2_p1_forward.wig', 'data_bacteroides/Subset_OnlyChrom\\ELP_S3_p1_forward.wig']
['data_bacteroides/Subset_OnlyChrom\\MLP_S4_p1_forward.wig', 'data_bacteroides/Subset_OnlyChrom\\MLP_S5_p1_forward.wig', 'data_bacteroides/Subset_OnlyChrom\\MLP_S6_p1_forward.wig']
['data_bacteroides/Subset_OnlyChrom\\Stat_S7_p1_forward.wig', 'data_bacteroides/Subset_OnlyChrom\\Stat_S8_p1_forward.wig', 'data_bacteroides/Subset_OnlyChrom\\Stat_S9_p1_forward.wig']


In [8]:
masterTable = pd.read_csv("data_bacteroides/published_genome.CSV", sep = ";")

In [9]:
masterTable = masterTable.dropna(subset=['Genome'])

In [10]:
masterTable

,SuperPos,Genome,detected,enriched,Pos,Strand,Locus_tag,Product,UTRlength,GeneLength,Sequence -50 nt upstream + TSS (51nt)
0,4778.0,EL_condition1,1.0,1.0,4778.0,+,BT_0008,hypothetical protein,180.0,546.0,AACCCCTTGACCGTTTGTGTATTTGCATAAATAGCATTACCTTTGC...
1,4778.0,ML_condition2,1.0,1.0,4778.0,+,BT_0008,hypothetical protein,180.0,546.0,AACCCCTTGACCGTTTGTGTATTTGCATAAATAGCATTACCTTTGC...
2,4778.0,SP_condition3,1.0,1.0,4778.0,+,BT_0008,hypothetical protein,180.0,546.0,AACCCCTTGACCGTTTGTGTATTTGCATAAATAGCATTACCTTTGC...
3,18773.0,SP_condition3,1.0,1.0,18773.0,+,BT_0022,hypothetical protein,271.0,291.0,TTATTTGCTTTGTAAATAGTAAAATCATAAATCAAATTTGTTATGG...
4,33886.0,EL_condition1,1.0,1.0,33886.0,+,BT_0037,transcriptional regulatory protein,201.0,1056.0,AGAAGAAAGCAAAGTGCTTATTATCAGATTGTTTTATTCACATGTA...
...,...,...,...,...,...,...,...,...,...,...,...
13168,6212611.0,ML_condition2,0.0,0.0,6212611.0,-,orphan,orphan,NaN,NaN,AAATGAAGTCAAACCTTTGAAAAACAAGGAATTTTATGTAACTTTA...
13169,6212611.0,SP_condition3,1.0,1.0,6212611.0,-,orphan,orphan,NaN,NaN,AAATGAAGTCAAACCTTTGAAAAACAAGGAATTTTATGTAACTTTA...
13170,6229025.0,EL_condition1,1.0,1.0,6229025.0,-,orphan,orphan,NaN,NaN,ATAGATGGCAGTATATTTTGCTATATTCTAAATATTGCCTACATTT...
13171,6229025.0,ML_condition2,1.0,1.0,6229025.0,-,orphan,orphan,NaN,NaN,ATAGATGGCAGTATATTTTGCTATATTCTAAATATTGCCTACATTT...


In [11]:
df_fordward["Stat"]

,value,zscore,first gradient,second gradient,previous
34,1.0,67.45,0.5,-0.25,0.0
91,1.0,67.45,0.5,-0.25,0.0
110,1.0,67.45,0.5,0.25,1.0
111,2.0,134.90,0.5,-0.25,1.0
158,2.0,134.90,0.5,0.25,2.0
...,...,...,...,...,...
6250067,1.0,67.45,0.5,0.25,1.0
6250068,2.0,134.90,0.5,-0.25,1.0
6250416,1.0,67.45,0.5,-0.25,0.0
6253180,1.0,67.45,0.5,-0.25,0.0


In [12]:
condition_EL = masterTable[masterTable["Genome"].str.contains("EL")]
condition_EL = condition_EL[condition_EL["detected"] == 1]
condition_EL = condition_EL[condition_EL["Strand"] == "+"]
condition_ML = masterTable[masterTable["Genome"].str.contains("ML")]
condition_ML = condition_ML[condition_ML["detected"] == 1]
condition_ML = condition_ML[condition_ML["Strand"] == "+"]
condition_SP = masterTable[masterTable["Genome"].str.contains("SP")]
condition_SP = condition_SP[condition_SP["detected"] == 1]
condition_SP = condition_SP[condition_SP["Strand"] == "+"]

In [13]:
tss_truth = {}
tss_truth["ELP"] = list(set(condition_EL["SuperPos"].values))
tss_truth["MLP"] = list(set(condition_ML["SuperPos"].values))
tss_truth["Stat"] = list(set(condition_SP["SuperPos"].values))

In [14]:
for condition in conditions:
    truth_indices = tss_truth[condition]

    labels = []
    for index in df_fordward[condition].index.values:
        if(index in truth_indices):
            labels += [1]
        else:
            labels += [0]
    df_fordward[condition]["label"] = labels

In [15]:
df_fordward["ELP"]["label"].value_counts()

label
0    232642
1       354
Name: count, dtype: int64

In [16]:
dfs = []
for df in df_fordward:
    dfs += [df_fordward[df]]

In [17]:
df = pd.concat(dfs, ignore_index=True)

In [18]:
df

,value,zscore,first gradient,second gradient,previous,label
0,1.0,67.45,0.5,-0.25,0.0,0
1,1.0,67.45,0.5,-0.25,0.0,0
2,2.0,134.90,1.5,7.50,0.0,0
3,3.0,202.35,16.0,7.00,2.0,0
4,34.0,2293.30,15.5,-8.00,3.0,0
...,...,...,...,...,...,...
600877,1.0,67.45,0.5,0.25,1.0,0
600878,2.0,134.90,0.5,-0.25,1.0,0
600879,1.0,67.45,0.5,-0.25,0.0,0
600880,1.0,67.45,0.5,-0.25,0.0,0


In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X = df.drop('label', axis=1)  # Features

In [21]:
y = df['label']  # Labels

In [22]:
df["label"].value_counts()

label
0    599779
1      1103
Name: count, dtype: int64

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=111)

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
clf = RandomForestClassifier(max_depth=10, random_state=111)

In [26]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=111)

In [27]:
y_pred = clf.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

In [29]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    197944
           1       0.78      0.39      0.52       348

    accuracy                           1.00    198292
   macro avg       0.89      0.69      0.76    198292
weighted avg       1.00      1.00      1.00    198292

[[197906     38]
 [   213    135]]


# Reverse evaluation on Bacteroides 

In [30]:
pattern_reverse = os.path.join(directory, '*reverse*')

In [31]:
files = glob.glob(pattern_reverse)

df_reverse = {}

for condition in conditions:
    file_list = []
    for file_path in files:
        if(condition in str(file_path)):
            file_list += [file_path]
    print(file_list)
    df_reverse[condition] = ops.parse_for_prediction(file_list, reverse= True)

['data_bacteroides/Subset_OnlyChrom\\ELP_S1_P1_reverse.wig', 'data_bacteroides/Subset_OnlyChrom\\ELP_S2_p1_reverse.wig', 'data_bacteroides/Subset_OnlyChrom\\ELP_S3_p1_reverse.wig']
['data_bacteroides/Subset_OnlyChrom\\MLP_S4_p1_reverse.wig', 'data_bacteroides/Subset_OnlyChrom\\MLP_S5_p1_reverse.wig', 'data_bacteroides/Subset_OnlyChrom\\MLP_S6_p1_reverse.wig']
['data_bacteroides/Subset_OnlyChrom\\Stat_S7_p1_reverse.wig', 'data_bacteroides/Subset_OnlyChrom\\Stat_S8_p1_reverse.wig', 'data_bacteroides/Subset_OnlyChrom\\Stat_S9_p1_reverse.wig']


In [32]:
condition_EL = masterTable[masterTable["Genome"].str.contains("EL")]
condition_EL = condition_EL[condition_EL["detected"] == 1]
condition_EL = condition_EL[condition_EL["Strand"] == "-"]
condition_ML = masterTable[masterTable["Genome"].str.contains("ML")]
condition_ML = condition_ML[condition_ML["detected"] == 1]
condition_ML = condition_ML[condition_ML["Strand"] == "-"]
condition_SP = masterTable[masterTable["Genome"].str.contains("SP")]
condition_SP = condition_SP[condition_SP["detected"] == 1]
condition_SP = condition_SP[condition_SP["Strand"] == "-"]

In [33]:
tss_truth = {}
tss_truth["ELP"] = list(set(condition_EL["SuperPos"].values))
tss_truth["MLP"] = list(set(condition_ML["SuperPos"].values))
tss_truth["Stat"] = list(set(condition_SP["SuperPos"].values))

In [34]:
for condition in conditions:
    truth_indices = tss_truth[condition]

    labels = []
    for index in df_reverse[condition].index.values:
        if(index in truth_indices):
            labels += [1]
        else:
            labels += [0]
    df_reverse[condition]["label"] = labels

In [35]:
df_reverse

{'ELP':          value  zscore  first gradient  second gradient  previous  label
 0          0.0     0.0             0.0              0.0       0.0      0
 1          0.0     0.0             0.0              0.0       0.0      0
 2          0.0     0.0             0.0              0.0       0.0      0
 3          0.0     0.0             0.0              0.0       0.0      0
 4          0.0     0.0             0.0              0.0       0.0      0
 ...        ...     ...             ...              ...       ...    ...
 6228546    0.0     0.0             0.0              0.0       0.0      0
 6228547    0.0     0.0             0.0              0.0       0.0      0
 6228548    0.0     0.0             0.0              0.0       0.0      0
 6228549    0.0     0.0             0.0              0.0       0.0      0
 6228550    0.0     0.0             0.0              0.0       0.0      0
 
 [6228551 rows x 6 columns],
 'MLP':          value  zscore  first gradient  second gradient  previous 

In [36]:
dfs = []
for df in df_reverse:
    dfs += [df_reverse[df]]

In [37]:
df_reverse = pd.concat(dfs, ignore_index=True)

In [38]:
df_reverse

,value,zscore,first gradient,second gradient,previous,label
0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...
18724059,0.0,0.0,0.0,0.0,0.0,0
18724060,0.0,0.0,0.0,0.0,0.0,0
18724061,0.0,0.0,0.0,0.0,0.0,0
18724062,0.0,0.0,0.0,0.0,0.0,0


In [39]:
X = df_reverse.drop('label', axis=1)  # Features

In [40]:
y = df_reverse['label']  # Labels

In [41]:
df_reverse["label"].value_counts()

label
0    18719858
1        4206
Name: count, dtype: int64

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=111)

In [43]:
y_pred = clf.predict(X_test)

In [44]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   6177541
           1       0.00      0.00      0.00      1401

    accuracy                           1.00   6178942
   macro avg       0.50      0.50      0.50   6178942
weighted avg       1.00      1.00      1.00   6178942

[[6176552     989]
 [   1397       4]]


In [45]:
clf_reverse = RandomForestClassifier(max_depth=10, random_state=111)

In [46]:
clf_reverse.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=111)

In [47]:
y_pred = clf_reverse.predict(X_test)

In [48]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   6177541
           1       0.80      0.35      0.48      1401

    accuracy                           1.00   6178942
   macro avg       0.90      0.67      0.74   6178942
weighted avg       1.00      1.00      1.00   6178942

[[6177417     124]
 [    915     486]]


In [49]:
df_reverse = df_reverse[df_reverse["first gradient"] < 0]
df_reverse = df_reverse[df_reverse["zscore"] > 0]

In [50]:
X = df_reverse.drop('label', axis=1)  # Features

In [51]:
y = df_reverse['label']  # Labels

In [52]:
clf_reverse = RandomForestClassifier(max_depth=10, random_state=111)

In [53]:
clf_reverse.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=111)

In [54]:
y_pred = clf_reverse.predict(X_test)

In [55]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   6177541
           1       0.80      0.35      0.48      1401

    accuracy                           1.00   6178942
   macro avg       0.90      0.67      0.74   6178942
weighted avg       1.00      1.00      1.00   6178942

[[6177417     124]
 [    915     486]]


In [56]:
import pickle

with open('../app/prediction/RandomForestClassifierTSSFilteredForward.pkl','wb') as f:
    pickle.dump(clf,f)
with open('../app/prediction/RandomForestClassifierTSSFilteredReverse.pkl','wb') as f:
    pickle.dump(clf_reverse,f)